In [ ]:
import numpy as np
from collections import defaultdict
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.file_locations import intermediate_files_location


# GENIE

In [ ]:
genie_all_knob_values = defaultdict(list)

universe_nums = []
knob_numbers = []
knob_names = []
knob_values = []

for filenum in range(6):
    # Downloaded from Nitish, /exp/uboone/data/users/bnayak/validation/genie_univs
    with open(f"../small_data_files/genie_univs/genie_{filenum}.txt", "r") as f:
        lines = f.readlines()
        if filenum == 0: lines = lines[8000:] # Prints out some weird values at the start of the first file before going into the real loop
        for line in lines:
            if "In universe" in line:
                if "RESRootino" in line:
                    continue

                universe_num = int(line.split("#")[1].split(",")[0])
                knob_number = int(line.split("#")[2].split(" ")[0])
                knob_name = line.split("(")[1].split(")")[0].strip()
                knob_value = float(line.split("value ")[1].strip())

                universe_nums.append(universe_num)
                knob_numbers.append(knob_number)
                knob_names.append(knob_name)
                knob_values.append(knob_value)

                genie_all_knob_values[knob_name].append(knob_value)

print(f"{'knob name':<30} {'num values':<15} {'first 5 values':<30}")
for knob_name, knob_values in genie_all_knob_values.items():
    print(f"{knob_name:<30} {len(knob_values):<15} {str(knob_values[:5])}")

# turn genie_all_knob_valuesinto a pandas dataframe
genie_knobs_df = pd.DataFrame(genie_all_knob_values)

genie_knobs_df.to_csv("../small_data_files/genie_knobs.csv", index=False)

genie_knobs_df

# Flux and Reinteraction

In [ ]:
# Modified from a version from Nitish, https://www.phy.bnl.gov/~nitish/uboone/clhep_random.cpp
"""
#include <iostream>
#include "CLHEP/Random/RandGaussQ.h"
#include "CLHEP/Random/Random.h"
#include "CLHEP/Random/RandomEngine.h"
#include "CLHEP/Random/JamesRandom.h"
#include "CLHEP/Random/RandFlat.h"

int main(int argc, char* argv[]) {
    // Check if a seed is provided as a command-line argument

    long seed = 12345;
    CLHEP::HepJamesRandom engine(seed);

    seed = 2;
    std::cout << "piplus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 3;
    std::cout << "piminus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 4;
    std::cout << "kplus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 5;
    std::cout << "kzero, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 6;
    std::cout << "kminus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 7;
    std::cout << "horncurrent, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 8;
    std::cout << "pioninexsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 9;
    std::cout << "nucleontotxsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 10;
    std::cout << "nucleonqexsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 11;
    std::cout << "nucleoninexsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 12;
    std::cout << "pionqexsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 13;
    std::cout << "piontotxsec, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 14;
    std::cout << "expskin, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;




    seed = 57;
    std::cout << "reinteractions_proton, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 58;
    std::cout << "reinteractions_piplus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 59;
    std::cout << "reinteractions_piminus, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 1000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    seed = 60;
    std::cout << "reinteractions_all, using seed " << seed << ": ";
    engine.setSeed(seed);
    for (int i = 0; i < 3000; i++) {
        double randomNumber = CLHEP::RandGaussQ::shoot(&engine, 0, 1);
        std::cout << randomNumber << ", ";
    }
    std::cout << std::endl;

    return 0;
}
"""

# setup_sl7
# cd /exp/uboone/app/users/lhagaman/
# g++ clhep_random.cpp -o clhep_random -lCLHEP -I`clhep-config --prefix | sed -e 's/"//g'`/include -L`clhep-config --prefix | sed -e 's/"//g'`/lib
# ./clhep_random 2

# flux seeds: https://github.com/uboone/ubsim/blob/develop/ubsim/EventWeight/jobs/flux/eventweight_microboone_flux.fcl

# reinteraction seeds: https://github.com/uboone/ubsim/blob/develop/ubsim/EventWeight/jobs/reint/eventweight_microboone_reint.fcl



In [ ]:
with open(f"../small_data_files/flux_reint_knob_outputs.txt", "r") as f:
    lines = f.readlines()

flux_reint_knob_values = {}

flux_reint_knob_output = []
for line in lines:
    if not line:
        continue
    knob_name = line.split(",")[0].strip()
    knob_values = [float(x) for x in line.split(":")[1].strip().split(",")[:-1]]
    print(knob_name, len(knob_values), knob_values[:5])
    flux_reint_knob_values[knob_name] = knob_values


# Validation

In [ ]:
weights_df = pl.read_parquet(f"{intermediate_files_location}/small_presel_weights_df.parquet")
weights_df

In [ ]:
def get_weight_sums(weights_df, weight_name):
    weights = np.stack(weights_df[weight_name].to_numpy())
    weights = np.nan_to_num(weights, nan=1, posinf=1, neginf=1)
    if weight_name == "All_UBGenie":
        weights[weights > 30] = 1
        weights[weights < 0] = 1
    weight_sums = np.sum(weights, axis=0)
    return weight_sums

#for energy_range in [(0, 500), (500, 1000), (1000, 2000), (2000, 1e9)]:
for energy_range in [(0, 1e9)]:

    fig, axes = plt.subplots(10, 5, figsize=(20, 30))
    axes = axes.flatten()
    num_knobs = len(genie_knobs_df.columns)
    for idx, knob in tqdm(enumerate(genie_knobs_df.columns), total=num_knobs):
        ax = axes[idx]
        ax.scatter(
            genie_knobs_df[knob][:500], 
            get_weight_sums(weights_df.filter((pl.col("filetype") == "nu_overlay") & (pl.col("wc_kine_reco_Enu") < energy_range[1]) & (pl.col("wc_kine_reco_Enu") > energy_range[0])), "All_UBGenie"), 
            s=2
        )
        ax.set_xlabel(knob)
        ax.set_ylabel("Sum of GENIE Weights")
    for ax in axes[num_knobs:]:
        ax.axis('off')
    plt.tight_layout()
    if energy_range == (0, 1e9):
        plt.suptitle(f"500 GENIE knob values in WC generic selected nu_overlay files", fontsize=30, y=1.03)
    else:
        plt.suptitle(f"500 GENIE knob values in WC generic selected nu_overlay files\nwith {energy_range[0]} < wc_kine_reco_Enu < {energy_range[1]}", fontsize=30, y=1.03)
    plt.show()


In [ ]:
for energy_range in [(0, 1e9)]:
    flux_knobs = [knob_name for knob_name in list(flux_reint_knob_values.keys()) if not "reinteractions" in knob_name]
    num_plots = len(flux_knobs)
    num_cols = 4
    num_rows = 4
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5 * num_cols, 5 * num_rows))
    axes = axes.flatten()
    for idx, flux_knob_name in tqdm(enumerate(flux_knobs), total=num_plots):
        ax = axes[idx]
        flux_knob_values = flux_reint_knob_values[flux_knob_name]
        yvals = get_weight_sums(
            weights_df.filter(
                (pl.col("filetype") == "nu_overlay") & 
                (pl.col("wc_kine_reco_Enu") < energy_range[1]) & 
                (pl.col("wc_kine_reco_Enu") > energy_range[0])
            ), 
            "flux_all"
        )
        ax.scatter(flux_knob_values, yvals, s=3)
        ax.set_xlabel(flux_knob_name)
        ax.set_ylabel("Sum of flux Weights")
    # Hide any unused axes
    for ax in axes[num_plots:]:
        ax.axis('off')
    plt.tight_layout()
    plt.suptitle("Flux knob values in WC generic selected nu_overlay files", fontsize=20, y=1.02)
    plt.show()

In [ ]:
reint_knobs

In [ ]:
for energy_range in [(0, 1e9)]:
    reint_knobs = [knob_name for knob_name in list(flux_reint_knob_values.keys()) if "reinteractions" in knob_name and "reinteractions_all" not in knob_name]
    num_plots = len(reint_knobs)
    num_cols = 2
    num_rows = 2
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5 * num_cols, 5 * num_rows))
    axes = axes.flatten()
    for idx, reint_knob_name in tqdm(enumerate(reint_knobs), total=num_plots):
        ax = axes[idx]
        reint_knob_values = flux_reint_knob_values[reint_knob_name]
        yvals = get_weight_sums(
            weights_df.filter(
                (pl.col("filetype") == "nu_overlay") & 
                (pl.col("wc_kine_reco_Enu") < energy_range[1]) & 
                (pl.col("wc_kine_reco_Enu") > energy_range[0])
            ), 
            "reint_all"
        )
        ax.scatter(reint_knob_values, yvals, s=3)
        ax.set_xlabel(reint_knob_name)
        ax.set_ylabel("Sum of reint Weights")
    # Hide any unused axes
    for ax in axes[num_plots:]:
        ax.axis('off')
    plt.tight_layout()
    plt.suptitle("Reinteraction knob values in WC generic selected nu_overlay files", fontsize=20, y=1.02)
    plt.show()